In [37]:
import os
from dotenv import load_dotenv
load_dotenv()

<function dotenv.main.load_dotenv(dotenv_path: Union[str, os.PathLike, NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [38]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [39]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)


In [40]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, I am Anil. I study B.Tech Final Year, CSE")])


AIMessage(content="Hi Anil,\n\nIt's great to meet you! \n\nBeing in your final year of B.Tech in CSE is a big accomplishment. What are you most looking forward to after graduation? Do you have any specific areas of computer science that you're particularly interested in?  \n\nI'm here to help if you have any questions about technology, need help brainstorming ideas, or just want to chat. 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 24, 'total_tokens': 114, 'completion_time': 0.163636364, 'prompt_time': 0.000138829, 'queue_time': 0.01288389, 'total_time': 0.163775193}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b15e35d3-d259-420f-a9e3-b972f12f1d9c-0', usage_metadata={'input_tokens': 24, 'output_tokens': 90, 'total_tokens': 114})

In [41]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, I am Anil. I study B.Tech Final Year, CSE"),
        AIMessage(content="Hey Anil!\n\nThat's awesome!  B.Tech in CSE is a challenging but rewarding field. \n\nWhat are you most looking forward to in your final year? Are you focusing on any specific areas of computer science?\n\nI'm happy to chat about anything related to your studies, tech in general, or even just life as a final year student.  😊\n"),
        HumanMessage(content="Can you please answer what I am pursuing?")
    ]
)

AIMessage(content='You are pursuing a Bachelor of Technology (B.Tech) degree in Computer Science Engineering (CSE).  \n\nIs there anything else I can help you with?\n', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 121, 'total_tokens': 156, 'completion_time': 0.063636364, 'prompt_time': 0.003802237, 'queue_time': 0.009664265, 'total_time': 0.067438601}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-adfa5fb1-cccc-4afe-b7a2-eac1704d9d72-0', usage_metadata={'input_tokens': 121, 'output_tokens': 35, 'total_tokens': 156})

### Message History
We can use a message history class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in data store. Future interactions will then load those messages and pass them into the chain as part of the input.

In [42]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id : str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [43]:
config = {"configurable":{"session_id":"chat1"}}

In [44]:
with_message_history.invoke(
    [
        HumanMessage(content="Hi, I am Anil. I study B.Tech Final Year, CSE"),
    ],
    config = config,
)

AIMessage(content="Hi Anil!\n\nThat's great!  B.Tech in Computer Science Engineering is a challenging and rewarding field. \n\nWhat are you most interested in within CSE? Are you focusing on a particular area like software development, artificial intelligence, cybersecurity, or something else?\n\nI'm here to help if you have any questions or need assistance with anything related to CSE or your studies.  😊\n", response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 24, 'total_tokens': 108, 'completion_time': 0.152727273, 'prompt_time': 0.00014723, 'queue_time': 0.013656228, 'total_time': 0.152874503}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9a55a81b-3add-44e1-b8ab-22f2dbcfce52-0', usage_metadata={'input_tokens': 24, 'output_tokens': 84, 'total_tokens': 108})

In [45]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Hi, I am Anil. I study B.Tech Final Year, CSE"),
    ],
    config = config,
)

response.content

"Hi Anil,\n\nIt's great to meet you!  Final year of B.Tech is a big milestone. \n\nWhat are you most looking forward to after graduation? Are you planning on pursuing a Master's degree, starting your career in the industry, or something else entirely?  \n\nI'm here to chat if you want to talk about your plans, or just anything else on your mind!\n"

In [46]:
with_message_history.invoke(
    [
        HumanMessage(content="What's my name?"),
    ],
    config = config,
)

AIMessage(content='Your name is Anil!  I remember you telling me at the beginning of our conversation. 😊  \n\nIs there anything else I can help you with?\n', response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 232, 'total_tokens': 266, 'completion_time': 0.061818182, 'prompt_time': 0.00914817, 'queue_time': 0.004188892999999999, 'total_time': 0.070966352}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8b87e707-3221-41e7-a8e8-d8ee22c3d4be-0', usage_metadata={'input_tokens': 232, 'output_tokens': 34, 'total_tokens': 266})

### Trying by changing the config file

In [47]:
### changed config file

config2 = {"configurable":{"session_id":"chat2"}}


In [48]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config2
)

### Here we can see that there is no data about Anil as we have changed the configuration i.e session_id

AIMessage(content="As an AI, I have no memory of past conversations and don't know your name. If you'd like to tell me, I'm happy to learn it! 😊  What's your name?\n", response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 15, 'total_tokens': 62, 'completion_time': 0.085454545, 'prompt_time': 7.416e-05, 'queue_time': 0.014271829, 'total_time': 0.085528705}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5eec752a-f8b5-4035-8097-75a7d699bf20-0', usage_metadata={'input_tokens': 15, 'output_tokens': 47, 'total_tokens': 62})

In [49]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

AIMessage(content='Your name is Anil!  \n\nIs there anything else I can help you with? 😊  \n', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 280, 'total_tokens': 302, 'completion_time': 0.04, 'prompt_time': 0.010334964, 'queue_time': 0.003248273999999999, 'total_time': 0.050334964}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8b575677-549d-4a98-9183-dfe652e90ce4-0', usage_metadata={'input_tokens': 280, 'output_tokens': 22, 'total_tokens': 302})

### Prompt Templates

Prompt templates help to run the raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First add in a system message with a custom instructions(but still taking messages as input). Next, we will add in more inputs just beside the messages.

In [53]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Please answer the questions asked as best as you know"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt_template|model

In [54]:
chain.invoke({"messages":[HumanMessage(content="Hi I am Anil")]})

AIMessage(content="Hi Anil! It's nice to meet you. \n\nWhat can I do for you today?  😊  \n", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 29, 'total_tokens': 56, 'completion_time': 0.049090909, 'prompt_time': 0.00013251, 'queue_time': 0.013605058, 'total_time': 0.049223419}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-532a7266-f34f-46bd-a11d-c6da08f95442-0', usage_metadata={'input_tokens': 29, 'output_tokens': 27, 'total_tokens': 56})

In [55]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [56]:
config3 = {"configurable":{"session_id":"chat3"}}

In [57]:
response = with_message_history.invoke([HumanMessage(content="Who Am I?")],config=config3)
response

AIMessage(content="As an AI, I don't have a personal identity like humans do. You could think of me as a computer program designed to understand and respond to your questions in a helpful and informative way.\n\nEssentially, I am a large language model here to assist you with tasks like:\n\n* **Providing information:** I can answer your questions based on the vast dataset I was trained on.\n* **Generating text:** I can create stories, poems, articles, and more.\n* **Summarizing text:** I can condense large amounts of information into shorter summaries.\n* **Translating languages:** I can translate text between different languages.\n\nIs there anything specific you'd like to know about me or would you like me to help you with something?\n", response_metadata={'token_usage': {'completion_tokens': 156, 'prompt_tokens': 29, 'total_tokens': 185, 'completion_time': 0.283636364, 'prompt_time': 0.000140309, 'queue_time': 0.015293141, 'total_time': 0.283776673}, 'model_name': 'Gemma2-9b-It', 's

In [58]:
response = with_message_history.invoke([HumanMessage(content="Hello, I am Anil?")],config=config3)
response

AIMessage(content="Hello Anil, it's nice to meet you! \n\nIs there anything I can help you with today? 😊  \n", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 199, 'total_tokens': 227, 'completion_time': 0.050909091, 'prompt_time': 0.010388805, 'queue_time': 0.0034180540000000002, 'total_time': 0.061297896}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c8bb2553-ed48-42bc-aa10-17af8bbd641e-0', usage_metadata={'input_tokens': 199, 'output_tokens': 28, 'total_tokens': 227})

In [59]:
response = with_message_history.invoke([HumanMessage(content="Who Am I?")],config=config3)
response

AIMessage(content="That's a great question!  Since you asked me, I can only tell you what you've told me so far:\n\nYou are Anil. \n\nIs there anything else you'd like to tell me about yourself? Perhaps your hobbies, where you're from, or what you like to do? \n\nThe more you tell me, the better I can understand who you are! 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 239, 'total_tokens': 327, 'completion_time': 0.16, 'prompt_time': 0.00873059, 'queue_time': 0.005697880000000001, 'total_time': 0.16873059}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8a8c14bd-17a4-4e19-94c0-e19b9acdb981-0', usage_metadata={'input_tokens': 239, 'output_tokens': 88, 'total_tokens': 327})

In [62]:
### Customizing the prompt template by adding a specific language translation
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Please answer the questions asked as best as you know in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt_template|model

In [75]:
response = chain.invoke({"messages":[HumanMessage(content="What is AI")],"language":"Hindi"})
response.content

'एआई यानी **कृत्रिम बुद्धिमत्ता**  \n\nकल्पना करो, एक मशीन जो इंसानों की तरह सोच सकती है, सीख सकती है और समस्याओं का हल कर सकती है।  यह  ही कृत्रिम बुद्धिमत्ता है। \n\nयह एक ऐसा क्षेत्र है जहाँ कंप्यूटर और रोबोट्स को मानव बुद्धिमत्ता जैसी क्षमताएं देने की कोशिश की जाती है। \n\nउदाहरण के लिए, \n\n* **वॉयस असिस्टेंट**: जैसे कि Google Assistant या Alexa\n* **फ़ोटो रिकॉग्निशन**: Facebook पर चेहरे की पहचान\n* **मशीन ट्रांसलेशन**: Google Translate \n\nये सभी कृत्रिम बुद्धिमत्ता के उदाहरण हैं। \n\n\nमुझे उम्मीद है कि यह समझ में आया होगा! क्या आप इसके बारे में और जानना चाहते हैं?\n'

Let's now wrap this more complicated chain in a Message History Class. Because, there are multiple keys in the input. We need to specify the correct keys to save and use the chat History

In [76]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [79]:
config4 = {"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="Hello! I am Anil")],"language":"Telugu"},
    config=config4
)

response.content

'హలో అనిల్ గారు! 😊 మీరు ఏంటి అడుగుతున్నారు? \n'

In [80]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="What's my name?")],"language":"Telugu"},
    config=config4
)

response.content

'మీ పేరు అనిల్ గారు.  😊 \n'

### Managing the Conversation History

One important concept to understand when building chat bots is managing conversation history. If left unmanaged, the list of messages will grow in number and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of messages when you are passing in.

trim_messages: Helper to reduce how many messages we are sending to the model. This trimmer allows us to specify how many tokens we want to keep, along with the parameters like if we want to always keep the system message and whether allow to make partial messages.  

In [92]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens = 50,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content="You are a good assistant"),
    HumanMessage(content="Hi! I'm Bob"),
    SystemMessage(content="Hi"),
    HumanMessage(content="I like vanilla Ice cream!"),
    SystemMessage(content="nice"),
    HumanMessage(content="what is 2+2"),
    SystemMessage(content="4"),
    HumanMessage(content="Thanks"),
    SystemMessage(content="No problem"),
    HumanMessage(content = "Having fun"),
    SystemMessage(content = "Yes")
]

trimmer.invoke(messages)

[SystemMessage(content='You are a good assistant'),
 HumanMessage(content='I like vanilla Ice cream!'),
 SystemMessage(content='nice'),
 HumanMessage(content='what is 2+2'),
 SystemMessage(content='4'),
 HumanMessage(content='Thanks'),
 SystemMessage(content='No problem'),
 HumanMessage(content='Having fun'),
 SystemMessage(content='Yes')]

In [93]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens = 30,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content="You are a good assistant"),
    HumanMessage(content="Hi! I'm Bob"),
    SystemMessage(content="Hi"),
    HumanMessage(content="I like vanilla Ice cream!"),
    SystemMessage(content="nice"),
    HumanMessage(content="what is 2+2"),
    SystemMessage(content="4"),
    HumanMessage(content="Thanks"),
    SystemMessage(content="No problem"),
    HumanMessage(content = "Having fun"),
    SystemMessage(content = "Yes")
]

trimmer.invoke(messages)

[SystemMessage(content='You are a good assistant'),
 HumanMessage(content='Thanks'),
 SystemMessage(content='No problem'),
 HumanMessage(content='Having fun'),
 SystemMessage(content='Yes')]

In [94]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens = 40,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content="You are a good assistant"),
    HumanMessage(content="Hi! I'm Bob"),
    SystemMessage(content="Hi"),
    HumanMessage(content="I like vanilla Ice cream!"),
    SystemMessage(content="nice"),
    HumanMessage(content="what is 2+2"),
    SystemMessage(content="4"),
    HumanMessage(content="Thanks"),
    SystemMessage(content="No problem"),
    HumanMessage(content = "Having fun"),
    SystemMessage(content = "Yes")
]

trimmer.invoke(messages)

[SystemMessage(content='You are a good assistant'),
 HumanMessage(content='what is 2+2'),
 SystemMessage(content='4'),
 HumanMessage(content='Thanks'),
 SystemMessage(content='No problem'),
 HumanMessage(content='Having fun'),
 SystemMessage(content='Yes')]

In [95]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (RunnablePassthrough.assign(messages = itemgetter("messages")|trimmer)|prompt_template|model)

response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What Ice Cream do I like?")], "language":"Telugu"
    }
)

response.content

### The AI doesn't know the output because I have reduced the tokens from the initial ones. 

' నేను మీకు తెలియదు! మీకు నచ్చే ఐస్ క్రీం రుచి ఏమిటో చెప్పండి.  🍦\n'

In [98]:
response = chain.invoke(
    {
        "messages":messages + [HumanMessage(content="What problem I have asked you?")], "language":"English"
    }
)
response.content
### gives output because the content is present in the reduced token size

'You asked me "What is 2+2?".  \n\nIs there anything else I can help you with? 😊  \n'